By: Michał Marusiński

# **For the best experience I advise to open this file in Google Colab!**


In [1]:
import pandas as pd
import numpy as np

In [3]:
#data_pic = pd.read_excel(r"C:\Users\48660\Desktop\Development Workshop\keys\participants_pic.xlsx")
data_pic = pd.read_excel(r"/content/participants_pic.xlsx")
data_pic.head()

,Numer osoby,płeć,kolor oczu,kalibracja_PICTURE1,kalibracja_PICTURE2,kalibracja_PICTURE3,Dodatkowe uwagi
0,S01,k,zielone,NaN,NaN,NaN,NaN
1,S02,k,NaN,3.096; 0.215,2.819; 0.220,3.015; 0.220,NaN
2,S03,k,piwne,3.227; 0.156,2.674; 0.156,2.575; 0.136,"czujnik światła był przysłonięty, powtórzona r..."
3,S04,k,piwne,3.137; 0.222,2.129; 0.204,2.924; 0.207,NaN
4,S05,m,piwne,2.400; 0.183,2.786; 0.178,2.381; 0.173,NaN


checking what type of comments are present in column 'Dodatkowe uwagi'

In [4]:
unique_comments = data_pic['Dodatkowe uwagi'].unique()
print(len(unique_comments))
print(f"\n\n {unique_comments}")

79


 [nan 'czujnik światła był przysłonięty, powtórzona rampa i video'
 'Duże problemy z kalibracją, badanie opóźnione o godzinę. Nie rozpoznaje źrenicy od tęczówki w momencie powiększenia źrenicy.'
 'Idealnie, szybko i sprawnie.'
 'Ludzie z głęboko osadzonymi oczami dużo gorzej się kalibrują, na części z obrazkami niby kalibracja ok a znajdywało potem bardziej całą tęczówkę\xa0niż źrenicę.'
 'Non-monotonic timestamp. Audio puszczone od nowa.'
 'non-monotonic timestamp - stracone dane z 5 ostatnich filmów'
 'głęboko osadzone oczy, ale nie ma tragedii'
 'druga część dźwięków w osobnym pliku'
 'non monotonic timestamp, rampa puszczona jeszcze raz'
 'trzy razy non-monotonic timestamp, programy włączone od nowa i dźwięk puszczony od początku'
 'non-monotonic timestamp w 1cz dżwięków, stracone 2-3 bodźce'
 'Pani robi kwestionariusze po badaniu bo nie zrobiła w domu'
 'były wyciszone dźwięki, poleciały od nowa w audio2'
 'Non-monotonic timestamp, jeden obrazek stracony, nagrywanie wznowione

In [6]:
data_pic[~data_pic['Dodatkowe uwagi'].isna()].shape

(88, 7)

We've got 88 samples with comments included, but only 79 unique comments are there. Some of them are duplicates and we'll try to figure out what comment is most frequent. It will simplify possible unqualifing patients for future processing

In [7]:
data_pic['Dodatkowe uwagi'] = data_pic['Dodatkowe uwagi'].str.lower()   #switching all letters to lower to interrupt counting the same comments as different becasue of letter type
data_pic[~data_pic['Dodatkowe uwagi'].isna()]['Dodatkowe uwagi'].value_counts()


Dodatkowe uwagi
wszystko ok                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        7
ok                                                                                                                                                                                                                                                                                                                                 

We see that some of the comments are positive one (wszystko ok / oki, ok) so there is no need to classify these patients as non-valueable
We delete all the sample's comments that has them positive giving the NaN value inplace

In [8]:
data_pic.loc[data_pic['Dodatkowe uwagi'].isin(['wszystko ok', 'ok', 'wszystko oki']), 'Dodatkowe uwagi'] = np.nan

In [9]:
data_pic[~data_pic['Dodatkowe uwagi'].isna()]['Dodatkowe uwagi'].value_counts()

Dodatkowe uwagi
wszystko ok                                                                                                                                                2
nie zatrzymało się nagrywanie po ostatnim bodźcu. restart programów, nie ma logów z pupil capture                                                          2
czujnik światła był przysłonięty, powtórzona rampa i video                                                                                                 1
non monotonic timestamp zaraz po przerwie w dzwiekach oraz w obrazkach                                                                                     1
non-monotonic timestamp na 1 cz obrazków, 6 straconych, puszczona od nowa 2 część z trzecią osobno (plik s88_pic3)                                         1
                                                                                                                                                          ..
kwestionariusze robione na miejscu        

In [10]:
a = data_pic.loc[data_pic['Dodatkowe uwagi'] == 'czujnik światła był przysłonięty, powtórzona rampa i video', 'Numer osoby'].to_numpy()[0]
type(a)

str

In [11]:
import time
from IPython.display import clear_output
comment = 0
invalid_samples = {}
for comment in data_pic[~data_pic['Dodatkowe uwagi'].isna()]['Dodatkowe uwagi'].unique():
  decision = 100
  print(comment)
  while not (decision == 1 or decision == 0):
      decision = int(input("Does this comment make the sample valid? 1 - yes / 0 - no \t"))
      print("")
      print(decision)
      if decision != 1 and decision != 0:
          print("Invalid input given, try again")
      else:
          break
  row = data_pic.loc[data_pic['Dodatkowe uwagi'] == comment]
  if decision == 1:
      continue
      #row['Dodatkowe uwagi'] == np.nan
  elif decision == 0:
      data_pic.drop(row.index, inplace= True)
      for sample in row['Numer osoby'].to_numpy():
          invalid_samples[sample] = comment
  time.sleep(1)
  clear_output(wait = True)
  #%clear


powyłączane rzeczy przez michała (programy w tle)
Does this comment make the sample valid? 1 - yes / 0 - no 	1

1
duże źrenice z natury, więc będzie słabsza kalibracja ale ładnie pokazuje na literki
Does this comment make the sample valid? 1 - yes / 0 - no 	1

1
puszczam zestawy nr 1 dla vid, audio i nr 11 dla obrazków (dla 1 był błąd w memory)
Does this comment make the sample valid? 1 - yes / 0 - no 	1

1
non-monotonic timestamp trzy razy na rampie pomimo bycia w trybie samolotowym i restartu kompa, potem znowu na filmie 8 razy, restart kompa, dźwięki nie grają, puszczone obrazki, potem zadania na komputerze i powrót do dźwięków
Does this comment make the sample valid? 1 - yes / 0 - no 	1

1
vr działał bez zarzuty, za to pythin postanowił nie działać i puściłam procedury na komputerze stacjonarnym (pierwszy lewej)
Does this comment make the sample valid? 1 - yes / 0 - no 	1

1
na rampioe i filmach non-monotonic timestamp pomimo uruchomienia trybu samolotowego. restart komputera przed

In [18]:
invalid_df = pd.DataFrame({'Patient number' : list(invalid_samples.keys()), 'Comment' : list(invalid_samples.values())})
invalid_df.head(1000)

,Patient number,Comment
0,S09,"duże problemy z kalibracją, badanie opóźnione ..."
1,S07,ludzie z głęboko osadzonymi oczami dużo gorzej...
2,S15,non-monotonic timestamp - stracone dane z 5 os...
3,S28,"non-monotonic timestamp w 1cz dżwięków, straco..."
4,S12,"error przy dźwiękach, druga połowa się zapętli..."
5,S24,"wysoka potliwość dłoni i eda dobre, ale odklej..."
6,S35,"error, stracone 4 filmy"
7,S37,"spoko, ale głębokie oczka i na rampie serduszk..."
8,S38,non-monotonic timestamp w ostatniej części 9 z...
9,S43,"duże źrenice z natury - pani mówi, że bardzo s..."


In [20]:
data_pic.to_excel('participants_pic_filtered.xlsx', index=False)
from google.colab import files
files.download('participants_pic_filtered.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>